# Machine Learning - UFRJ 2022.2
## Regressão Logísitca, SVM e Naive Bayes

#### Feito por 
- Rafael da Silva Fernandes, DRE: 117196229
- Sidney Outeiro, DRE: 118201873

## Bibliotecas

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Necessário rodar para ler arquivos .xls no Colab
!pip install --upgrade xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Dataset e Tratamento de Dados
Trabalharemos com base em um dataset sobre os **passageiros do Titanic**, que pode ser encontrado em: https://www.kaggle.com/vinicius150987/titanic3.


In [ ]:
df = pd.read_excel('titanic3.xls')
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


### Dados do dataset:
- Total de instâncias: 1309
- Total de colunas: 14

### Descrição das colunas:
- pclass (int): Classe do ingresso (Primeira = 1, Segunda = 2 ou Terceira  = 3 classe)
- survived (int):
 - 0: Não sobreviveu
 - 1: Sobreviveu ao Titanic
- name (string): Nome do passageiro
- sex (string): Sexo do passageiro
 - female: Feminino
 - male: Masculino
- age (float): Idade do passageiro
- sibsp (int): 
 - 0: O passageiro não possui irmãos (siblings) ou parceiros (spouse) a bordo
 - 1: O passageiro possui irmãos (siblings) ou parceiros (spouse) a bordo
- parch (int):
 - 0: O passageiro não possui pais (parents) ou filhos (children) a bordo
 - 1: O passageiro não possui irmãos (parents) ou parceiros (children) a bordo
- ticket (string): Número do bilhete do passageiro
- fare(float): Valor da passagem da viagem (Libra Esterlina)
- cabin (string): Número da cabine do passageiro
- embarked (string):
 - C: O passageiro embarcou no porto de Cherbourg
 - Q: O passageiro embarcou no porto de Queenstown
 - S: O passageiro embarcou no porto de Southampton
 - nan: Não informado
- boat (string): Número de identificação do barco salva-vidas
- body (int): Número de identificação do cadáver
- home.dest (string): Origem / Destino do passageiro

### Tratamento de dados

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1309 non-null   int64  
 1   survived   1309 non-null   int64  
 2   name       1309 non-null   object 
 3   sex        1309 non-null   object 
 4   age        1046 non-null   float64
 5   sibsp      1309 non-null   int64  
 6   parch      1309 non-null   int64  
 7   ticket     1309 non-null   object 
 8   fare       1308 non-null   float64
 9   cabin      295 non-null    object 
 10  embarked   1307 non-null   object 
 11  boat       486 non-null    object 
 12  body       121 non-null    float64
 13  home.dest  745 non-null    object 
dtypes: float64(3), int64(4), object(7)
memory usage: 143.3+ KB


Podemos observar que as colunas cabin, boat, body têm menos do que a metade do total de instâncias do dataset sem questão. Essa é uma das razões para desconsiderarmos essas colunas nos nosso modelos. Outra razão é o fato de:
- cabin: diz respeito à cabine em que o passageiro estava alocado, mas a hora em que houve o trágico incidente do Titanic era um momento em que a grande maioria dos passageiros estavam foram de seus quartos ou para jantar ou para aproveitar o cruzeiro em si. Portanto, não é um atributo bom para treinarmos o nosso modelo.
- boat: diz respeito ao bote salva-vidas em que o passageiro embarcou. Como o passageiro embarcou no bote, podemos inferir que ele sobreviveu ao acidente. Portanto, não vamos treinar o nosso modelo com esse atributo.
- body: diz respeito ao número de identificação do cadáver, ou seja, 100% das instâncias que possuem o campo body preenchido se referem à pessoas que não sobreviveram, e portanto é um atributo que deve ser desconsiderado.
- home.dest: diz respeito à cidade de origem/ destino do passageiro. Visto que o acidente aconteceu em alto-mar, esse dado é irrelevante.

In [ ]:
df.drop(columns = ['cabin', 'boat', 'body', 'home.dest'], inplace = True)

Como foi observado na descrição da coluna embarked, há valores nan no dataset. Por essa razão, iremos removê-los, assim como valores que sejam iguais à infinito ou -infinito. Após esse tratamento, as instâncias do dataset caíram de 1309 para 1043.

In [ ]:
df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]

## Treinamento dos modelos de classificação

### Dividindo o dataset em treino e teste

Para o treinamento, consideraremos apenas as colunas pclass, sex, age, sibsp e parch, pois não acreditamos que o nome da  pessoa, o número e preço do seu ingresso, e em qual porto ela embarcou estejam relacionados ao fato da pessoa ter sobrevivido ou não ao naufrágio do navio.

Como os modelos recebem apenas valores numéricos, precisamos antes converter a coluna sex. Faremos com que "female" vire o valor 1, e portanto, "male" se tornará o valor 0.

In [ ]:
df['sex'] = np.where(df['sex'] == 'female', 1, 0)

In [ ]:
X = df[['pclass', 'sex', 'age', 'sibsp', 'parch']]
y = df['survived']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 1/3, random_state = 181022
)

### Regressão Logística

In [ ]:
logistic_reg = LogisticRegression(solver = 'liblinear')
logistic_reg.fit(X_train, y_train)
logistic_reg_score = logistic_reg.score(X_test, y_test)
logistic_reg_score

0.764367816091954

Os coeficientes positivos aumentam as probabilidades logarítmicas da resposta (e, portanto, aumentam a probabilidade), e os coeficientes negativos diminuem as probabilidades logarítmicas da resposta (e, portanto, diminuem a probabilidade):
- sex é o coeficiente positivo mais alto, implicando que à medida que o valor de Sexo aumenta (masculino: 0 a feminino: 1), a probabilidade de Sobrevivência = 1 aumenta.
- Inversamente à medida que pclass aumenta, a probabilidade de Sobrevivência = 1 diminui mais.


In [ ]:
coeff_df = pd.DataFrame(X_train.columns)
coeff_df.columns = ['Feature']
coeff_df['Correlation'] = pd.Series(logistic_reg.coef_[0])
coeff_df.sort_values(by = 'Correlation', ascending = False)

,Feature,Correlation
1,sex,2.548337
4,parch,0.020987
2,age,-0.030493
3,sibsp,-0.317195
0,pclass,-1.006366


### SVM

In [ ]:
svm_clf = SVC(gamma = 'auto')
svm_clf.fit(X_train, y_train)
svm_clf_score = svm_clf.score(X_test, y_test)
svm_clf_score

0.7816091954022989

### Linear SVM

In [ ]:
linear_svm_clf = LinearSVC()
linear_svm_clf.fit(X_train, y_train)
linear_svm_clf_score = linear_svm_clf.score(X_test, y_test)
linear_svm_clf_score

0.6235632183908046

### Naive Bayes

In [ ]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
gnb_score = gnb.score(X_test, y_test)
gnb_score

0.7787356321839081

## Comparação dos resultados

In [ ]:
models = pd.DataFrame({
    'Modelo': ['Regressão Logísitca', 'SVM', 'Linear SVM', 'Naive Bayes'],
    'Score': [logistic_reg_score, svm_clf_score, linear_svm_clf_score, gnb_score]
})

models.sort_values(by = 'Score', ascending = False)

,Modelo,Score
1,SVM,0.781609
3,Naive Bayes,0.778736
0,Regressão Logísitca,0.764368
2,Linear SVM,0.623563
